In [ ]:
!pip install yfinance

In [2]:
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
import requests

In [5]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", 
              columns=['Security', 
                       'GICS Sub-Industry',
                       'Headquarters Location', 
                       'Date added'])

In [6]:
df = df[['Symbol', 'Security', 'Headquarters Location', 'GICS Sub-Industry', 'Date added']]

In [7]:
df

,Symbol,Security,Headquarters Location,GICS Sub-Industry,Date added
0,MMM,3M,"Saint Paul, Minnesota",Industrial Conglomerates,1957-03-04
1,AOS,A. O. Smith,"Milwaukee, Wisconsin",Building Products,2017-07-26
2,ABT,Abbott,"North Chicago, Illinois",Health Care Equipment,1957-03-04
3,ABBV,AbbVie,"North Chicago, Illinois",Pharmaceuticals,2012-12-31
4,ACN,Accenture,"Dublin, Ireland",IT Consulting & Other Services,2011-07-06
...,...,...,...,...,...
498,YUM,Yum! Brands,"Louisville, Kentucky",Restaurants,1997-10-06
499,ZBRA,Zebra Technologies,"Lincolnshire, Illinois",Electronic Equipment & Instruments,2019-12-23
500,ZBH,Zimmer Biomet,"Warsaw, Indiana",Health Care Equipment,2001-08-07
501,ZION,Zions Bancorporation,"Salt Lake City, Utah",Regional Banks,2001-06-22


In [8]:
#df = df.merge(df_new, right_on='Symbol', left_on='Ticker')

In [9]:
#df = df.drop(columns=['Symbol'])

In [11]:
df = df.rename(columns={'Headquarters Location': 'Headquarters', 'GICS Sub-Industry': 'Industry'})

In [12]:
df

,Symbol,Security,Headquarters,Industry,Date added
0,MMM,3M,"Saint Paul, Minnesota",Industrial Conglomerates,1957-03-04
1,AOS,A. O. Smith,"Milwaukee, Wisconsin",Building Products,2017-07-26
2,ABT,Abbott,"North Chicago, Illinois",Health Care Equipment,1957-03-04
3,ABBV,AbbVie,"North Chicago, Illinois",Pharmaceuticals,2012-12-31
4,ACN,Accenture,"Dublin, Ireland",IT Consulting & Other Services,2011-07-06
...,...,...,...,...,...
498,YUM,Yum! Brands,"Louisville, Kentucky",Restaurants,1997-10-06
499,ZBRA,Zebra Technologies,"Lincolnshire, Illinois",Electronic Equipment & Instruments,2019-12-23
500,ZBH,Zimmer Biomet,"Warsaw, Indiana",Health Care Equipment,2001-08-07
501,ZION,Zions Bancorporation,"Salt Lake City, Utah",Regional Banks,2001-06-22


In [13]:
df.columns

Index(['Symbol', 'Security', 'Headquarters', 'Industry', 'Date added'], dtype='object')

In [17]:
stock_dict = {}
df_ticker_list = df.Symbol.unique()

In [ ]:
stock_dict['Competitors'] = []
stock_dict['Ticker'] = []
stock_dict['Average Volume'] = []

for ticker in df_ticker_list:
  try:
    stock_dict['Ticker'].append(ticker)
    print('ticker', ticker)
    request = requests.get('https://money.cnn.com/quote/quote.html?symb={}'.format(ticker))
    content = request.content
    soup = BeautifulSoup(content, "html.parser")

    stock_dict['Competitors'].append([soup.find_all("a", class_='wsod_symbol')[i].get_text() for i in range(len(soup.find_all("a", class_='wsod_symbol')))])
    stock_dict['Average Volume'].append(soup.find_all("td", class_='wsod_quoteDataPoint')[4].get_text())

  except:
    stock_dict['Ticker'].append(ticker)
    ticker = ticker.replace(".", "")
    request = requests.get('https://money.cnn.com/quote/quote.html?symb={}'.format(ticker))
    content = request.content
    soup = BeautifulSoup(content, "html.parser")

    stock_dict['Competitors'].append([soup.find_all("a", class_='wsod_symbol')[i].get_text() for i in range(len(soup.find_all("a", class_='wsod_symbol')))])
    stock_dict['Average Volume'].append(soup.find_all("td", class_='wsod_quoteDataPoint')[4].get_text())
    

In [41]:
df_updated = pd.DataFrame.from_dict(stock_dict)
df_updated

,Ticker,Average Volume,Competitors
0,MMM,"3,195,777",[]
1,AOS,"1,134,110","['SPXC', 'PFIE']"
2,ABT,"4,918,836","['BSX', 'BDX', 'DXCM', 'ZBH']"
3,ABBV,"5,651,585","['MRK', 'LLY', 'NVO', 'PFE']"
4,ACN,"2,160,135",['GIB']
...,...,...,...
498,YUM,"1,702,476","['CMG', 'YUMC', 'QSR', 'DRI']"
499,ZBRA,"416,844","['HPE', 'SMCI', 'TDC']"
500,ZBH,"1,389,282","['WST', 'SAUHF', 'BAX', 'PODD']"
501,ZION,"2,217,245","['BSAC', 'BOKF']"


In [44]:
df_final = df_updated.merge(df, right_on='Symbol', left_on='Ticker')

In [45]:
from google.colab import files

df_final.to_csv('df_stock_info.csv')
files.download('df_stock_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# New Section

In [ ]:
%%time
#this takes about 3 minutes

start_date = '2013-03-16'
end_date = '2023-03-16'
interval = '1d'
price_data = []

for ticker in df_ticker_list:
  #print('ticker', ticker)
  t = ticker
  ticker = yf.Ticker(ticker)
  data = ticker.history(start=start_date, end=end_date, interval=interval)
  data['Ticker'] = str(t)
  price_data.append(data)

price_data = pd.concat(price_data, ignore_index=False)